a)

In [ ]:
'''
    Multi-class classifier w/ neural nets
'''
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
#may not need
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i,sequence in enumerate(sequences):
        results[i,sequence] = 1
    return results

if "__main__" == __name__:

    best_params = {
        'layer_count': 0,
        'node_count': 0,
        'learning_rate': 0,
        'epochs': 0,
        'loss': 10000,
        'val_loss': 10000,
        'acc': 0,
        'val_acc': 0
    }

    best_loss_params = best_params
    best_acc_params = best_params

    # load data
    np_load_old = np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
    (X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=10000)
    word_index = reuters.get_word_index(path="reuters_word_index.json")
    np.load = np_load_old

    X_train = vectorize_sequences(X_train)
    X_test = vectorize_sequences(X_test)
    one_hot_y_train = to_categorical(y_train)
    one_hot_y_test = to_categorical(y_test)

    X_val = X_train[:1000]
    partial_X_train = X_train[1000:]
    y_val = one_hot_y_train[:1000]
    partial_y_train = one_hot_y_train[1000:]
    iteration = 0
    for hidden_layers in range(1, 6):
        for hidden_layer_nodes in [8, 16, 24, 32, 40, 48, 56, 64]:

                    model = Sequential()
                    model.add(Dense(64, activation='relu', input_shape=(10000,)))

                    for i in range(hidden_layers):
                        model.add(Dense(hidden_layer_nodes, activation='relu'))

                    model.add(Dense(46, activation='softmax'))

                    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

                    result_data = []
                    for learning_rate in [0.0001, 0.001, 0.01, 0.1]:
                        print("\n### Iteration:" + str(iteration) + " hidden layers:" + str(hidden_layers) + " hidden layer nodes:"
                            + str(hidden_layer_nodes) + " learning rate:" + str(learning_rate) + " ###\n")
                        iteration += 1
                        for num_epochs in [20, 40, 60, 80, 100]:
                            K.set_value(model.optimizer.learning_rate, learning_rate)
                            history = model.fit(partial_X_train, partial_y_train, epochs=num_epochs, batch_size=512, validation_data=(X_val, y_val), verbose=0)

                            loss = history.history['loss']
                            val_loss = history.history['val_loss']
                            acc = history.history['acc']
                            val_acc = history.history['val_acc']

                            epochs = range(1, len(loss) + 1)
                            result_data.append([num_epochs, loss[-1], val_loss[-1]])
                            if((best_params['loss'] + best_params['val_loss']) > (loss[-1] + val_loss[-1]) 
                                and (best_params['acc'] + best_params['val_acc']) < (acc[-1] + val_acc[-1])):
                                best_params = {
                                    'layer_count': hidden_layers,
                                    'node_count': hidden_layer_nodes,
                                    'learning_rate': learning_rate,
                                    'epochs': num_epochs,
                                    'loss': loss[-1],
                                    'val_loss': val_loss[-1],
                                    'acc': acc[-1],
                                    'val_acc': val_acc[-1]
                                }

                            if((best_loss_params['loss'] + best_loss_params['val_loss']) > (loss[-1] + val_loss[-1])):
                                best_loss_params = {
                                    'layer_count': hidden_layers,
                                    'node_count': hidden_layer_nodes,
                                    'learning_rate': learning_rate,
                                    'epochs': num_epochs,
                                    'loss': loss[-1],
                                    'val_loss': val_loss[-1],
                                    'acc': acc[-1],
                                    'val_acc': val_acc[-1]
                                }

                            if((best_acc_params['acc'] + best_acc_params['val_acc']) < (acc[-1] + val_acc[-1])):
                                best_acc_params = {
                                    'layer_count': hidden_layers,
                                    'node_count': hidden_layer_nodes,
                                    'learning_rate': learning_rate,
                                    'epochs': num_epochs,
                                    'loss': loss[-1],
                                    'val_loss': val_loss[-1],
                                    'acc': acc[-1],
                                    'val_acc': val_acc[-1]
                                }

                            print("\n### Best Params ###")
                            print("Overall: " + str(best_params))
                            print("Accuracy: " + str(best_acc_params))
                            print("Loss: " + str(best_acc_params))

                        # plt.clf()
                        # plt.plot([item[0] for item in result_data], [item[1] for item in result_data], 'r', label='Training Loss')
                        # plt.plot([item[0] for item in result_data], [item[2] for item in result_data], 'b', label='Validation Loss')
                        # plt.title('Training Loss vs Validation Loss')
                        # plt.xlabel('Epochs')
                        # plt.ylabel('Loss')
                        # plt.legend()
                        # plt.show()
                        result_data = [] 

    # plt.clf()
    # acc = history.history['acc']
    # val_acc = history.history['val_acc']
    # plt.plot(epochs, acc,'r',label='Training Accuracy')
    # plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    # plt.xlabel('Epochs')
    # plt.ylabel('Accuracy')
    # plt.legend()
    # plt.show()

    # results = model.evaluate(X_test, one_hot_y_test)
    # print(results)

/home/radbrad/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/radbrad/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/radbrad/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/radbrad/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

### Iteration:0 hidden layers:1 hidden layer nodes:8 learning rate:0.0001 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 20, 'loss': 1.8978197915426802, 'val_loss': 1.993780807495117, 'acc': 0.652468, 'val_acc': 0.625}
Accuracy: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 20, 'loss': 1.8978197915426802, 'val_loss': 1.993780807495117, 'acc': 0.652468, 'val_acc': 0.625}
Loss: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 20, 'loss': 1.8978197915426802, 'val_loss': 1.993780807495117, 'acc': 0.652468, 'val_acc': 0.625}

### Best Params ###
Overall: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 40, 'loss': 0.8782532800561174, 'val_loss': 1.2104271459579468, 'acc': 0.7744926, 'val_acc': 0.707}
Accuracy: {'layer_count': 1, 'node_count': 8, 'l


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.1853282613074442, 'val_loss': 1.239158224105835, 'acc': 0.95051366, 'val_acc': 0.769}
Accuracy: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.1853282613074442, 'val_loss': 1.239158224105835, 'acc': 0.95051366, 'val_acc': 0.769}
Loss: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.1853282613074442, 'val_loss': 1.239158224105835, 'acc': 0.95051366, 'val_acc': 0.769}

### Iteration:3 hidden layers:1 hidden layer nodes:8 learning rate:0.1 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.1853282613074442, 'val_loss': 1.239158224105835, 'acc': 0.95051366, 'val_acc': 0.769}
Accuracy: {'layer_count': 1, 'node_count': 8, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.1853282613074442, 'val_loss': 1.239158224105835, 'acc': 0.95051366, 


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 16, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.10398965407152637, 'val_loss': 1.205813648223877, 'acc': 0.96091205, 'val_acc': 0.802}
Accuracy: {'layer_count': 1, 'node_count': 16, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.10398965407152637, 'val_loss': 1.205813648223877, 'acc': 0.96091205, 'val_acc': 0.802}
Loss: {'layer_count': 1, 'node_count': 16, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.10398965407152637, 'val_loss': 1.205813648223877, 'acc': 0.96091205, 'val_acc': 0.802}

### Iteration:6 hidden layers:1 hidden layer nodes:16 learning rate:0.01 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 16, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.10398965407152637, 'val_loss': 1.205813648223877, 'acc': 0.96091205, 'val_acc': 0.802}
Accuracy: {'layer_count': 1, 'node_count': 16, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.10398965407152637, 'val_loss': 1.205813648223877, 'acc': 


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.1846058669730375, 'val_loss': 0.9604600553512573, 'acc': 0.9550238, 'val_acc': 0.815}
Accuracy: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc': 0.9605362, 'val_acc': 0.813}
Loss: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc': 0.9605362, 'val_acc': 0.813}

### Iteration:9 hidden layers:1 hidden layer nodes:24 learning rate:0.001 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.1846058669730375, 'val_loss': 0.9604600553512573, 'acc': 0.9550238, 'val_acc': 0.815}
Accuracy: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc': 


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.1846058669730375, 'val_loss': 0.9604600553512573, 'acc': 0.9550238, 'val_acc': 0.815}
Accuracy: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc': 0.9605362, 'val_acc': 0.813}
Loss: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc': 0.9605362, 'val_acc': 0.813}

### Iteration:12 hidden layers:1 hidden layer nodes:32 learning rate:0.0001 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.1846058669730375, 'val_loss': 0.9604600553512573, 'acc': 0.9550238, 'val_acc': 0.815}
Accuracy: {'layer_count': 1, 'node_count': 24, 'learning_rate': 0.0001, 'epochs': 80, 'loss': 0.08250816993973542, 'val_loss': 1.1245543174743653, 'acc'


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 32, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.14489399105407516, 'val_loss': 0.9253294982910156, 'acc': 0.9576547, 'val_acc': 0.822}
Accuracy: {'layer_count': 1, 'node_count': 32, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.14489399105407516, 'val_loss': 0.9253294982910156, 'acc': 0.9576547, 'val_acc': 0.822}
Loss: {'layer_count': 1, 'node_count': 32, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.14489399105407516, 'val_loss': 0.9253294982910156, 'acc': 0.9576547, 'val_acc': 0.822}

### Iteration:15 hidden layers:1 hidden layer nodes:32 learning rate:0.1 ###


### Best Params ###
Overall: {'layer_count': 1, 'node_count': 32, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.14489399105407516, 'val_loss': 0.9253294982910156, 'acc': 0.9576547, 'val_acc': 0.822}
Accuracy: {'layer_count': 1, 'node_count': 32, 'learning_rate': 0.0001, 'epochs': 60, 'loss': 0.14489399105407516, 'val_loss': 0.9253294982910156, 'acc':

In [ ]:
b)

In [ ]:
c)